In [3]:
import pandas as pd
import psycopg2
from io import StringIO
from utils.utils import connection

In [2]:
conn = connection()

In [ ]:
chunk_size = 10000
chunks = []

query = "SELECT * FROM t_pulse_data"

for chunk in pd.read_sql_query(query, conn, chunksize=chunk_size):
    chunks.append(chunk)

df_t_pulse = pd.concat(chunks, ignore_index=True)